<h2>Best Places to Open a Bar in Toronto</h2>

<h4>Import Libraries</h4>

In [73]:
from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<h3>Load Neighbourhood Data based on Postal Code </h3>

In [74]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')  
#load column headers
headers = []
i=0
for th in soup.find_all('th'):
    if i < 3:
        headers.insert(i, th.text.replace('\n','') )
    i +=1
print(headers)

['Postcode', 'Borough', 'Neighbourhood']


<h3>Load Postal Code Data into DataFrame</h3>

In [75]:
html_rows = soup.find('tbody')
y=0
out_rows = []
for tr in html_rows.find_all('tr'):
    x=0
    html_cols = []
    for td in tr.find_all('td'):
        html_cols.insert(x,td.text.replace('\n',''))
        x +=1
    if len(html_cols) > 0:
        #check if neighbourhood not assigned
        if html_cols[2] == 'Not assigned':
            html_cols[2] = html_cols[1]
        # are we on the first record?
        if y > 0 :
            # make sure the previous record exists
            if len(out_rows[y-1]) > 0:
                # check for same postal code
                if out_rows[y-1][0] == html_cols[0]:
                    out_rows[y-1][2] = out_rows[y-1][2] + ', ' + html_cols[2]
                else:
                    out_rows.insert(y,html_cols)
                    y +=1
            else:
                out_rows.insert(y,html_cols)
                y +=1
        else:
            out_rows.insert(y,html_cols)
            y +=1
df = pd.DataFrame(out_rows)
df.columns=headers
df = df[df.Borough != 'Not assigned']
df = df.reset_index(drop=True)
print(df.head(10))

  Postcode           Borough                     Neighbourhood
0      M3A        North York                         Parkwoods
1      M4A        North York                  Victoria Village
2      M5A  Downtown Toronto         Harbourfront, Regent Park
3      M6A        North York  Lawrence Heights, Lawrence Manor
4      M7A      Queen's Park                      Queen's Park
5      M9A         Etobicoke                  Islington Avenue
6      M1B       Scarborough                    Rouge, Malvern
7      M3B        North York                   Don Mills North
8      M4B         East York   Woodbine Gardens, Parkview Hill
9      M5B  Downtown Toronto          Ryerson, Garden District


In [28]:
df.shape

(103, 3)

<h3>Get the GeoData based on Postal Code Locations</h3>

In [76]:
locations = pd.read_csv("http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv")
locations.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<h3>Merge the GeoData with the Neighbourhood Data by Postal Code Locations</h3>

In [77]:
df = df.rename(columns={'Postcode': 'Postal Code'})
df_locations = pd.merge(df, locations, how='right', on='Postal Code')
df_locations = df_locations.rename(columns={'Latitude_x': 'Latitude','Longitude_y': 'Longitude'})
df_locations.head(11)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


<h3>Define Foursquare Credentials and Version</h3>

In [79]:
CLIENT_ID = 'MEKIUJ5XWI00QTXZM2WSNNAMF30CD44PLLEM54JQZP3IQPCK' # your Foursquare ID
CLIENT_SECRET = '1S13EASXVWSU2ESM1EWW1QJBP2V0VKFQUBHBVXIW4MTOKL4V' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

<h2>Explore Neighborhoods in Toronto</h2>

<h3>Create a function to repeat the same process to all the neighborhoods in Toronto</h3>

In [80]:
url = 'https://api.foursquare.com/v2/venues/categories?&client_id={}&client_secret={}&v={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION 
            )
results = requests.get(url).json()
name_list = []
i=0
for name in results['response']['categories']:
    name_list.append(results['response']['categories'][i]['name'])
    i +=1
print(name_list)
#'There are {} items.'.format(len(results['response']['categories']))

['Arts & Entertainment', 'College & University', 'Event', 'Food', 'Nightlife Spot', 'Outdoors & Recreation', 'Professional & Other Places', 'Residence', 'Shop & Service', 'Travel & Transport']


<h3>Run the above function on each neighborhood and create a new dataframe</h3>

In [81]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
   
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [82]:
radius = 500
LIMIT = 100
toronto_venues = getNearbyVenues(names=df_locations['Neighbourhood'],
                                   latitudes=df_locations['Latitude'],
                                   longitudes=df_locations['Longitude']
                                  )
print('Completed exploring all Neighbourhoods.')

Completed exploring all Neighbourhoods.


In [83]:
toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


<h3>Filter by bar venues only</h3>

In [84]:
bar_venues = toronto_venues[toronto_venues['Venue Category'].isin(['Brewery','Speakeasy','Sake Bar','Lounge','Jazz Club','Strip Club','Bistro','Irish Pub','Social Club','Gastropub','Bar','Beer Bar','Hookah Bar','Wine Bar','Cocktail Bar','Sports Bar','Piano Bar','Gay Bar','Pub','Nightclub'])]
bar_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
15,"Harbourfront, Regent Park",43.654260,-79.360636,Dominion Pub and Kitchen,43.656919,-79.358967,Pub
34,"Harbourfront, Regent Park",43.654260,-79.360636,Mill St. Brew Pub,43.650353,-79.358489,Pub
47,"Harbourfront, Regent Park",43.654260,-79.360636,Ontario Spring Water Sake Company,43.649922,-79.360073,Brewery
90,Queen's Park,43.662301,-79.389494,Buddies In Bad Times Theatre,43.663059,-79.383449,Nightclub
109,Queen's Park,43.662301,-79.389494,SUDS,43.659880,-79.394712,Bar


<h3>Check how many venues were returned for each neighborhood</h3>

In [85]:
bar_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",10,10,10,10,10,10
Agincourt,1,1,1,1,1,1
"Alderwood, Long Branch",1,1,1,1,1,1
"Bedford Park, Lawrence Manor East",1,1,1,1,1,1
Berczy Park,11,11,11,11,11,11
"Brockton, Exhibition Place, Parkdale Village",2,2,2,2,2,2
Business reply mail Processing Centre969 Eastern,1,1,1,1,1,1
"Cabbagetown, St. James Town",3,3,3,3,3,3
Central Bay Street,5,5,5,5,5,5


<h3>Analyze Each Neighborhood</h3>

In [86]:
# one hot encoding
bar_onehot = pd.get_dummies(bar_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
bar_onehot['Neighbourhood'] = bar_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [bar_onehot.columns[-1]] + list(bar_onehot.columns[:-1])
bar_onehot = bar_onehot[fixed_columns]

bar_onehot.head(10)

,Neighbourhood,Bar,Beer Bar,Bistro,Brewery,Cocktail Bar,Gastropub,Gay Bar,Hookah Bar,Irish Pub,Jazz Club,Lounge,Nightclub,Piano Bar,Pub,Sake Bar,Social Club,Speakeasy,Sports Bar,Strip Club,Wine Bar
15,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
34,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
47,"Harbourfront, Regent Park",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
90,Queen's Park,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
109,Queen's Park,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
119,"Woodbine Gardens, Parkview Hill",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
162,"Ryerson, Garden District",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
163,"Ryerson, Garden District",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
175,"Ryerson, Garden District",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
187,"Ryerson, Garden District",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [87]:
bar_onehot.shape

(203, 21)

<h3>Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category</h3>

In [88]:
bar_grouped = bar_onehot.groupby('Neighbourhood').mean().reset_index()
bar_grouped

,Neighbourhood,Bar,Beer Bar,Bistro,Brewery,Cocktail Bar,Gastropub,Gay Bar,Hookah Bar,Irish Pub,Jazz Club,Lounge,Nightclub,Piano Bar,Pub,Sake Bar,Social Club,Speakeasy,Sports Bar,Strip Club,Wine Bar
0,"Adelaide, King, Richmond",0.300000,0.000000,0.000000,0.000000,0.100000,0.200000,0.000000,0.000000,0.000000,0.100000,0.100000,0.000000,0.000000,0.000000,0.000000,0.0,0.100000,0.000000,0.000000,0.100000
1,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
2,"Alderwood, Long Branch",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
3,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
4,Berczy Park,0.000000,0.181818,0.090909,0.000000,0.272727,0.000000,0.000000,0.000000,0.090909,0.090909,0.090909,0.090909,0.000000,0.090909,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
5,"Brockton, Exhibition Place, Parkdale Village",0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
6,Business reply mail Processing Centre969 Eastern,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
7,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.666667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
8,Central Bay Street,0.600000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.200000
9,"Chinatown, Grange Park, Kensington Market",0.500000,0.100000,0.100000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.100000


In [89]:
bar_grouped.shape

(42, 21)

<h3>Print each neighbourhood along with the top 5 most common venues</h3>

In [90]:
num_top_venues = 5

for hood in bar_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = bar_grouped[bar_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
       venue  freq
0        Bar   0.3
1  Gastropub   0.2
2  Speakeasy   0.1
3  Jazz Club   0.1
4     Lounge   0.1


----Agincourt----
        venue  freq
0      Lounge   1.0
1    Beer Bar   0.0
2  Strip Club   0.0
3  Sports Bar   0.0
4   Speakeasy   0.0


----Alderwood, Long Branch----
        venue  freq
0         Pub   1.0
1         Bar   0.0
2    Beer Bar   0.0
3  Strip Club   0.0
4  Sports Bar   0.0


----Bedford Park, Lawrence Manor East----
        venue  freq
0         Pub   1.0
1         Bar   0.0
2    Beer Bar   0.0
3  Strip Club   0.0
4  Sports Bar   0.0


----Berczy Park----
          venue  freq
0  Cocktail Bar  0.27
1      Beer Bar  0.18
2        Lounge  0.09
3           Pub  0.09
4        Bistro  0.09


----Brockton, Exhibition Place, Parkdale Village----
        venue  freq
0         Bar   0.5
1   Nightclub   0.5
2    Beer Bar   0.0
3  Strip Club   0.0
4  Sports Bar   0.0


----Business reply mail Processing Centre969 Eastern----
        

<h3>Sort the venues in descending order.</h3>

In [91]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

<h3>Create a new dataframe and display the top 3 venues for each neighbourhood.</h3>

In [92]:
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = bar_grouped['Neighbourhood']

#print('bar',bar_grouped.head())
#print('ven',neighbourhoods_venues_sorted.head())

for ind in np.arange(bar_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bar_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,"Adelaide, King, Richmond",Bar,Gastropub,Lounge
1,Agincourt,Lounge,Wine Bar,Irish Pub
2,"Alderwood, Long Branch",Pub,Wine Bar,Irish Pub
3,"Bedford Park, Lawrence Manor East",Pub,Wine Bar,Irish Pub
4,Berczy Park,Cocktail Bar,Beer Bar,Jazz Club
5,"Brockton, Exhibition Place, Parkdale Village",Bar,Nightclub,Irish Pub
6,Business reply mail Processing Centre969 Eastern,Brewery,Wine Bar,Strip Club
7,"Cabbagetown, St. James Town",Pub,Gastropub,Wine Bar
8,Central Bay Street,Bar,Gastropub,Wine Bar
9,"Chinatown, Grange Park, Kensington Market",Bar,Cocktail Bar,Beer Bar


<h3>Put the neighbourhoods into 4 Clusters</h3>

In [93]:
# set number of clusters
kclusters = 4

bar_grouped_clustering = bar_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bar_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 3, 3, 0, 2, 1, 3, 2, 2], dtype=int32)

In [94]:
bar_merged = df_locations

bar_merged_new = bar_merged[bar_merged['Neighbourhood'].isin(neighbourhoods_venues_sorted['Neighbourhood'])]
bar_merged_new = bar_merged_new.reindex()
bar_merged_new['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
bar_merged_new = bar_merged_new.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

bar_merged_new.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,0,Pub,Brewery,Wine Bar
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,0,Bar,Nightclub,Irish Pub
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937,3,Gastropub,Wine Bar,Strip Club
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,3,Bar,Pub,Beer Bar
10,M6B,North York,Glencairn,43.709577,-79.445073,0,Pub,Wine Bar,Irish Pub


<h2>Examination of the Clusters</h2>

<h3>The Downtown Neighbourhoods are a very popular place for higher end types of bars.</h3>  

In [104]:
bar_merged_new.loc[bar_merged_new['Cluster Labels'] == 0, bar_merged_new.columns[[1,2,8,7,6]]]

,Borough,Neighbourhood,3rd Most Common Venue,2nd Most Common Venue,1st Most Common Venue
2,Downtown Toronto,"Harbourfront, Regent Park",Wine Bar,Brewery,Pub
4,Queen's Park,Queen's Park,Irish Pub,Nightclub,Bar
10,North York,Glencairn,Irish Pub,Wine Bar,Pub
24,Downtown Toronto,Central Bay Street,Wine Bar,Gastropub,Bar
25,Downtown Toronto,Christie,Irish Pub,Wine Bar,Nightclub
30,Downtown Toronto,"Adelaide, King, Richmond",Lounge,Gastropub,Bar
36,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",Bar,Brewery,Sports Bar
42,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",Bar,Gastropub,Sports Bar
48,Downtown Toronto,"Commerce Court, Victoria Hotel",Cocktail Bar,Bar,Gastropub
65,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",Strip Club,Wine Bar,Brewery


<h3>The West End of Toronto Neighbourhoods are popular for seedier types of bars.</h3>  

In [96]:
bar_merged_new.loc[bar_merged_new['Cluster Labels'] == 1, bar_merged_new.columns[[1,2,8,7,6]]]

,Borough,Neighbourhood,3rd Most Common Venue,2nd Most Common Venue,1st Most Common Venue
15,Downtown Toronto,St. James Town,Beer Bar,Gastropub,Cocktail Bar
31,West Toronto,"Dovercourt Village, Dufferin",Strip Club,Brewery,Bar
37,West Toronto,"Little Portugal, Trinity",Pub,Cocktail Bar,Bar


<h3>The Neighbourhoods on the outskirts of Toronto are a popular for live music bars.</h3>  

In [97]:
bar_merged_new.loc[bar_merged_new['Cluster Labels'] == 2, bar_merged_new.columns[[1,2,8,7,6]]]

,Borough,Neighbourhood,3rd Most Common Venue,2nd Most Common Venue,1st Most Common Venue
12,Scarborough,"Highland Creek, Rouge Hill, Port Union",Beer Bar,Strip Club,Bar
20,Downtown Toronto,Berczy Park,Jazz Club,Beer Bar,Cocktail Bar
23,East York,Leaside,Wine Bar,Gastropub,Brewery
41,East Toronto,"The Danforth West, Riverdale",Lounge,Pub,Brewery
47,East Toronto,"The Beaches West, India Bazaar",Wine Bar,Pub,Brewery
55,North York,"Bedford Park, Lawrence Manor East",Irish Pub,Wine Bar,Pub
59,North York,Willowdale South,Irish Pub,Wine Bar,Lounge
74,Central Toronto,"The Annex, North Midtown, Yorkville",Irish Pub,Wine Bar,Pub
75,West Toronto,"Parkdale, Roncesvalles",Irish Pub,Piano Bar,Bar
78,Scarborough,Agincourt,Irish Pub,Wine Bar,Lounge


<h3>Some Neighbourhoods around Toronto host fewer numbers of popular casual bars.</h3>  

In [98]:
bar_merged_new.loc[bar_merged_new['Cluster Labels'] == 3, bar_merged_new.columns[[1,2,8,7,6]]]

,Borough,Neighbourhood,3rd Most Common Venue,2nd Most Common Venue,1st Most Common Venue
8,East York,"Woodbine Gardens, Parkview Hill",Strip Club,Wine Bar,Gastropub
9,Downtown Toronto,"Ryerson, Garden District",Beer Bar,Pub,Bar
19,East Toronto,The Beaches,Irish Pub,Wine Bar,Pub
34,North York,"Northwood Park, York University",Beer Bar,Strip Club,Bar
43,West Toronto,"Brockton, Exhibition Place, Parkdale Village",Irish Pub,Nightclub,Bar
54,East Toronto,Studio District,Brewery,Bar,Gastropub
93,Etobicoke,"Alderwood, Long Branch",Irish Pub,Wine Bar,Pub
96,Downtown Toronto,"Cabbagetown, St. James Town",Wine Bar,Gastropub,Pub
97,Downtown Toronto,"First Canadian Place, Underground city",Pub,Gastropub,Bar


<h3>Load the Wellbeing information</h3> 

In [118]:
wellbeing = pd.read_csv("wellbeing_toronto.csv")
wellbeing['Total'] = (wellbeing['After-Tax Household Income'] + wellbeing['Pop 15 - 64 years'] - (wellbeing['Robberies'] * 450))/100
wellbeing.head()

,Neighbourhood,Neighbourhood Id,After-Tax Household Income,Pop 15 - 64 years,Robberies,Total
0,West Humber-Clairville,1,59703,23285,82,460.88
1,Mount Olive-Silverstone-Jamestown,2,46986,22300,81,328.36
2,Thistletown-Beaumond Heights,3,57522,6760,12,588.82
3,Rexdale-Kipling,4,51194,7165,15,516.09
4,Elms-Old Rexdale,5,49425,6370,14,494.95


<h3>Load the Toronto Neighbourhood geo codes</h3>

In [99]:
with open('TORONTO_NEIGHBOURHOODS.geojson') as json_data:
    toronto_geo = json.load(json_data)

#print(toronto_geo)
print('GeoJSON file uploaded!')

GeoJSON file uploaded!


<h3>Map the Toronto Neighbourhoods, color code the neighbourhoods by score (Crime, Poplulation, Income)</h3>

In [120]:
# define the map centered around Toronto with a low zoom level
toronto_map = folium.Map(location=[43.667505488, -79.312476109], zoom_start=11)
# create a numpy array of length 6 and has linear spacing from the minium total immigration to the maximum total immigration
threshold_scale = np.linspace(wellbeing['Total'].min(),
                              wellbeing['Total'].max(),
                              6, dtype=int)
threshold_scale = threshold_scale.tolist() # change the numpy array to a list
threshold_scale[-1] = threshold_scale[-1] + 1 # make sure that the last value of the list is greater than the maximum immigration

# generate choropleth map, color code by Wellbeing (crime, population, income) 
toronto_map.choropleth(
    geo_data=toronto_geo,
    data=wellbeing,
    columns=['Neighbourhood', 'Total'],
    key_on='feature.properties.AREA_NAME',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Wellbeing in Toronto',
    threshold_scale=threshold_scale,
    reset=True
)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
cluster_type = ['Higher End Bars','Seedier Bars','Live Music Bars','Casual Bars' ]

# add existing popular bars as markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bar_merged_new['Latitude'], bar_merged_new['Longitude'], bar_merged_new['Neighbourhood'], bar_merged_new['Cluster Labels']):
    label = folium.Popup( str(poi) + ': ' + cluster_type[cluster], parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(toronto_map)
    
# add neighbourhood markers to the map
#for lat, lon, poi in zip(df_locations['Latitude'], df_locations['Longitude'], df_locations['Neighbourhood']):
#    label = folium.Popup( str(poi) , parse_html=True)
#    folium.Marker(
#        [lat, lon],
#        popup=label).add_to(toronto_map)
    
# display map
toronto_map



<h2>Conclusion</h2>

The lightest shaded neighbourhoods offer the safest communities with a large affluent population. The darkest (red) neighbourhoods offer the least in safety, population and affluence.  Based on the map above, there are some clear options on locations to open a new bar. They are:
    
    1. Lawrence Park and York Mills are of the lightest shaded neighbourhoods (in the Borough of East York) and there are no popular bars currently in that area. An entrepreneur willing to be the first kid in town, with an upscale bar<, may do well there.
    
    2. The neighbourhoods of Dovercourt Village and Cristie are also of the lightest shaded neighbourhoods. There are seedier type bars and upscale bars in the area so a casual bar or one with live music would do well there.
            
    3. The downtown neighbourhoods have a high population and affluence. Although there are many bars of many types, the Waterfront and Regent Park neighbourhoods do not have any popular bars. This would be a prime location for any type of bar.

